In [60]:
import pymysql.cursors
import pandas as pd
import numpy as np
import tqdm

tables_of_interests = [
    'unitandsynset',
    'synset',
    'lexicalunit',
    'lexicalrelation',
    'synsetrelation'
]

In [86]:
markednesses = {
    '- m': -1,
    '+ m': 1,
    '- s': -0.5,
    '+ s': 0.5
}

connection = pymysql.connect(host='db',
                             user='root',
                             password='password',
                             charset='utf8mb4',
                             database='wordnet_new',
                             cursorclass=pymysql.cursors.DictCursor)
try:
    with connection.cursor() as cursor:
        sql = """
        SELECT em.id, em.emotions, em.markedness, em.example1, em.example2, lex.lemma FROM `emotion` as em
        LEFT JOIN lexicalunit as lex on lex.ID = em.lexicalunit_id 
        WHERE em.emotions <> ''
        AND em.example1 <> ''
        AND em.markedness <> 'amb'
        """
        cursor.execute(sql)
        result = cursor.fetchall()
        print(len(result))
        parsed_res = [
            [data['emotions'], markednesses[data['markedness']] if data['markedness'] in markednesses else 0, data['example1'], data['example2'], data['lemma']]
            for data in result
        ]
#         parsed_res = [item for sublist in parsed_res for item in sublist]
        for data in result[:5]:
            print()
            for el, val in data.items():
                print(f'{el}: {val}')
finally:
    connection.close()

46824

id: 9
emotions: smutek;złość;strach;
markedness: - m
example1: Za swoje winy został skazany przez sąd na dożywocie bez możliwości wcześniejszego warunkowego zwolnienia. 
example2: 
lemma: dożywocie

id: 10
emotions: radość;cieszenie się na coś oczekiwanego;
markedness: + s
example1: W moim czasie wolnym lubię odpoczywać na działce i uprawiać sport. 
example2: 
lemma: czas wolny

id: 11
emotions: radość;cieszenie się na coś oczekiwanego;
markedness: + s
example1: Lubię mieć wolne od pracy i uczelni - mogę wtedy odpocząć i nabrać sił. 
example2: 
lemma: wolne

id: 30
emotions: cieszenie się na coś oczekiwanego;
markedness: + s
example1: Kibice kieleckiego beniaminka przyszli na pierwszy po awansie mecz pełni nadziei.
example2: 
lemma: beniaminek

id: 40
emotions: smutek;złość;
markedness: - s
example1: Zła passa nie opuszcza warszawskiego spadkowicza - właśnie przegrał kolejny mecz.
example2: 
lemma: spadkowicz


In [113]:
emotions_mapping = {
    '': '',
    '-': '', 
    'brzydota': 'wstret', 
    'bład': '', 
    'błąd': '', 
    'cieszenie sie na': 'radosc',
    'cieszenie się': 'radosc', 
    'cieszenie się na': 'radosc',
    'cieszenie się na coś oczekiwanego': 'oczekiwanie', 
    'dobro': 'radosc', 
    'gniew': 'zlosc', 
    'krzywda': 'zlosc',
    'nieszczęscie': 'smutek', 
    'nieszczęście': 'smutek', 
    'nieużyteczność': '', 
    'niewiedza': '',
    'piękno': '', 
    'prawda': '', 
    'radosć': 'radosc', 
    'radoć': 'radosc', 
    'radośc': 'radosc', 
    'radość': 'radosc',
    'radość: piękno': 'radosc', 
    'radość: szczęście': 'radosc', 
    's mutek': 'smutek', 
    'smitek': 'smutek',
    'smute': 'smutek', 
    'smutek': 'smutek', 
    'smutek: krzywda': 'smutek', 
    'strach': 'strach', 
    'strach wstręt': 'strach',
    'szczęście': 'radosc', 
    'użyteczność': '', 
    'wiedza': '', 
    'wstret': 'wstret', 
    'wstrę': 'wstret', 
    'wstręt': 'wstret',
    'wstręt: brzydota': 'wstret', 
    'wstręt: nieużyteczność': 'wstret', 
    'wstęt': 'wstret',
    'zaskoczenie': 'zaskoczenie', 
    'zaskoczenie czymś nieprzewidywanym': 'zaskoczenie', 
    'zaufanie': 'podziw',
    'zlość': 'zlosc', 
    'zufanie': 'podziw', 
    'złosć': 'zlosc', 
    'złośc': 'zlosc', 
    'złość': 'zlosc',
    'złość: nieużyteczność': 'zlosc', 
    'złóść': 'zlosc'
}

remove_empty_examples = []
for data in parsed_res:
    emotions = data[0].split(';')
    for emotion in emotions:
        emotion_mapped = emotions_mapping[emotion]
        if emotion_mapped != '':
            if data[2] != '' and data[2] is not None:
                remove_empty_examples.append([emotion_mapped, data[1], data[2], data[4]])
            if data[3] != '' and data[3] is not None:
                remove_empty_examples.append([emotion_mapped, data[1], data[3], data[4]])


In [114]:
len(remove_empty_examples)

80411

In [115]:
print(remove_empty_examples[0])
df = pd.DataFrame(remove_empty_examples, columns=[['emotions', 'markedness', 'text', 'lemma']])
df

['smutek', -1, 'Za swoje winy został skazany przez sąd na dożywocie bez możliwości wcześniejszego warunkowego zwolnienia. ', 'dożywocie']


,emotions,markedness,text,lemma
0,smutek,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie
1,zlosc,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie
2,strach,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie
3,radosc,0.5,W moim czasie wolnym lubię odpoczywać na dział...,czas wolny
4,oczekiwanie,0.5,W moim czasie wolnym lubię odpoczywać na dział...,czas wolny
...,...,...,...,...
80406,smutek,-1.0,Po diabelsku potraktowałeś tego człowieka - ni...,po diabelsku
80407,zlosc,-1.0,Po diabelsku potraktowałeś tego człowieka - ni...,po diabelsku
80408,wstret,-1.0,Po diabelsku potraktowałeś tego człowieka - ni...,po diabelsku
80409,wstret,-0.5,"A zupka dzisiaj jakaś taka cieniuteńka, jakby ...",cieniuteńki


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80411 entries, 0 to 80410
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (emotions,)    80411 non-null  object 
 1   (markedness,)  80411 non-null  float64
 2   (text,)        80411 non-null  object 
 3   (lemma,)       80326 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.5+ MB


In [123]:
df1 = df.dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80326 entries, 0 to 80410
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (emotions,)    80326 non-null  object 
 1   (markedness,)  80326 non-null  float64
 2   (text,)        80326 non-null  object 
 3   (lemma,)       80326 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [124]:
df1.to_csv('../data/slowosiec_data.csv',index=False)

In [125]:
emotions = [row[0] for row in remove_empty_examples]
np.unique(emotions)

array(['oczekiwanie', 'podziw', 'radosc', 'smutek', 'strach', 'wstret',
       'zaskoczenie', 'zlosc'], dtype='<U11')